## google drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## library import

In [ ]:
# bayesian optimization 설치
!pip install hyperopt

In [ ]:
import numpy as np, os, pandas as pd, sys
from hyperopt import fmin, hp, tpe
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.layers import Dense, LSTM, BatchNormalization
from keras.models import save_model, Sequential
from keras.optimizers import AdamW
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

## Constants

In [ ]:
# 데이터 경로, 변수 등 수정하여 사용할 변수(하이퍼파라미터는 아래 bayesian에서 설정)

DATA_PATH = "/content/drive/MyDrive/숨고/신유진님(LSTM bayesian optimization)/data/"
MODEL_PATH = "/content/drive/MyDrive/숨고/신유진님(LSTM bayesian optimization)/result/model/"
GRAPH_PATH = "/content/drive/MyDrive/숨고/신유진님(LSTM bayesian optimization)/result/graph/"
METRIC_RESULT_PATH = "/content/drive/MyDrive/숨고/신유진님(LSTM bayesian optimization)/result/metric/"
INPUT_COLUMN = ["TEMP_AVG","PRECI_DAY","POP","BUSINESS","RATIO","PRICE"]
TARGET_COLUMN = "HOUSING_E"
TRAIN_RATE = 0.8
VALIDATION_RATE=0.1
SEQ_LEN = 6
EPOCHS = 300
PATIENCE = 30
MAX_EVALS = 3

## data load

In [ ]:
# 불러온 DATA_PATH로 데이터 목록 저장
data_list = os.listdir(DATA_PATH)
print("data_list :", data_list)

# data명을 key, data를 value로 하는 딕셔너리 생성
data_dict = {}
for data_name in data_list:
    data_dict[data_name.split(".")[0]] = pd.read_csv(DATA_PATH+data_name)

data_list : ['SIG_CD_28110.csv']


In [ ]:
data_dict

{'SIG_CD_28110':      SIG_CD     SD SGG    TIME  YEAR  MONTH   HOUSING_E   TEMP_AVG  PRECI_DAY  \
 0     28110  인천광역시  중구  200801  2008      1   9201.7842  -2.207293   0.240370   
 1     28110  인천광역시  중구  200802  2008      2   9130.5615  -1.462737   0.111990   
 2     28110  인천광역시  중구  200803  2008      3   8173.5771   5.983078   1.191049   
 3     28110  인천광역시  중구  200804  2008      4   8241.7100  11.961664   1.363898   
 4     28110  인천광역시  중구  200805  2008      5   7726.3359  15.747392   1.561367   
 ..      ...    ...  ..     ...   ...    ...         ...        ...        ...   
 151   28110  인천광역시  중구  202008  2020      8  21723.7950  25.259909  10.486750   
 152   28110  인천광역시  중구  202009  2020      9  22150.0310  20.747995   6.011920   
 153   28110  인천광역시  중구  202010  2020     10  16869.8730  14.210952   0.046633   
 154   28110  인천광역시  중구  202011  2020     11  17771.9320   7.926172   1.619568   
 155   28110  인천광역시  중구  202012  2020     12  19156.7790   0.164155   0.143381   


## data preprocess
- 아래 과정을 모든 행정구역 데이터에 대해 반복
    - train/validation/test 분리
    - minmaxscaling 수행
    - lstm 학습을 위한 전처리

In [ ]:
# lstm에 맞게 데이터 처리하는 함수
def sequential_processing(data, window_size, input_columns, target_column):
    X = data[INPUT_COLUMN]
    y = data[[TARGET_COLUMN]]

    input = []
    output = []
    output_time = []

    for i in range(len(data) - window_size):
        input.append(X.iloc[i:i+window_size])
        output.append(y.iloc[i+window_size])
        output_time.append(data.iloc[i+window_size,3:4])

    return input, output, output_time

data_preprocessed_dict = {}
sub_data_dict = {}
#  및 lstm에 맞게 전처리 수행 후, train/validation/test 분리 및 스케일링
for key,value in data_dict.items():

    # sequential 전처리
    X_data_list, y_data_list, y_time_list = sequential_processing(value, SEQ_LEN, INPUT_COLUMN, TARGET_COLUMN)

    # train/test 분리
    X_train_list = X_data_list[:round(len(X_data_list)*TRAIN_RATE)]
    y_train_list = y_data_list[:round(len(y_data_list)*TRAIN_RATE)]
    y_train_time_list = y_time_list[:round(len(y_time_list)*TRAIN_RATE)]
    X_test_list = X_data_list[round(len(X_data_list)*TRAIN_RATE):]
    y_test_list = y_data_list[round(len(y_data_list)*TRAIN_RATE):]
    y_test_time_list = y_time_list[round(len(y_time_list)*TRAIN_RATE):]

    # validation의 경우 train set에서 VALIDATION_RATE 만큼 추출
    X_validation_list = X_train_list[round(len(X_train_list)*(1-VALIDATION_RATE)):]
    y_validation_list = y_train_list[round(len(y_train_list)*(1-VALIDATION_RATE)):]
    y_validation_time_list = y_train_time_list[round(len(y_train_time_list)*(1-VALIDATION_RATE)):]
    X_train_list = X_train_list[:round(len(X_train_list)*(1-VALIDATION_RATE))]
    y_train_list = y_train_list[:round(len(y_train_list)*(1-VALIDATION_RATE))]
    y_train_time_list = y_train_time_list[:round(len(y_train_time_list)*(1-VALIDATION_RATE))]

    # train set의 input column들에 대해 minmaxscaling 수행 후, validation, test는 transform 진행
    # train list를 행방향 병합한 다음, minmaxscaler를 만들고 train, validation, test 리스트의 각 데이터프레임에 대해 transform 수행
    tmp_X_train_df = pd.concat(X_train_list,axis=0)
    mm_scaler = MinMaxScaler()
    mm_scaler = mm_scaler.fit(tmp_X_train_df)

    # train, validation, test 리스트의 각 데이터프레임에 대해 transform 수행
    for i in range(len(X_train_list)):
        X_train_list[i] = pd.DataFrame(mm_scaler.transform(X_train_list[i]), columns=INPUT_COLUMN)

    for i in range(len(X_validation_list)):
        X_validation_list[i] = pd.DataFrame(mm_scaler.transform(X_validation_list[i]), columns=INPUT_COLUMN)

    for i in range(len(X_test_list)):
        X_test_list[i] = pd.DataFrame(mm_scaler.transform(X_test_list[i]), columns=INPUT_COLUMN)

    sub_data_dict['X_train'] = np.array(X_train_list)
    sub_data_dict['y_train'] = np.array(y_train_list)
    sub_data_dict['y_train_time'] = np.array(y_train_time_list)
    sub_data_dict['X_validation'] = np.array(X_validation_list)
    sub_data_dict['y_validation'] = np.array(y_validation_list)
    sub_data_dict['y_validation_time'] = np.array(y_validation_time_list)
    sub_data_dict['X_test'] = np.array(X_test_list)
    sub_data_dict['y_test'] = np.array(y_test_list)
    sub_data_dict['y_test_time'] = np.array(y_test_time_list)
    data_preprocessed_dict[key] = sub_data_dict

    print(f"{key} 행정구역에 대한 처리 결과" + "="*30)
    print("train input shape :", sub_data_dict['X_train'].shape)
    print("train label shape :", sub_data_dict['y_train'].shape)
    print("train time shape :", sub_data_dict['y_train_time'].shape)
    print("validation input shape :", sub_data_dict['X_validation'].shape)
    print("validation label shape :", sub_data_dict['y_validation'].shape)
    print("validation time shape :", sub_data_dict['y_validation_time'].shape)
    print("test input shape :", sub_data_dict['X_test'].shape)
    print("test label shape :", sub_data_dict['y_test'].shape)
    print("test time shape :", sub_data_dict['y_test_time'].shape)

SIG_CD_28110 행정구역에 대한 처리 결과==============================
train input shape : (108, 6, 6)
train label shape : (108, 1)
train time shape : (108, 1)
validation input shape : (12, 6, 6)
validation label shape : (12, 1)
validation time shape : (12, 1)
test input shape : (30, 6, 6)
test label shape : (30, 1)
test time shape : (30, 1)


## LSTM optimization process
- 아래 과정을 모든 행정구역 데이터에 대해 반복
    - bayesian optimization을 통해 각 행정구역별 LSTM 최적화

In [ ]:
total_result = pd.DataFrame()
total_feature_importance = pd.DataFrame()
for key in data_preprocessed_dict:

    X_train = data_preprocessed_dict[key]['X_train']
    y_train = data_preprocessed_dict[key]['y_train']
    X_validation = data_preprocessed_dict[key]['X_validation']
    y_validation = data_preprocessed_dict[key]['y_validation']
    X_test = data_preprocessed_dict[key]['X_test']
    y_test = data_preprocessed_dict[key]['y_test']
    time_train = data_preprocessed_dict[key]['y_train_time']
    time_validation = data_preprocessed_dict[key]['y_validation_time']
    time_test = data_preprocessed_dict[key]['y_test_time']

    # RMSE 정의
    def RMSE(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

    # LSTM 모델 정의
    def lstm_model(params):
        model = Sequential()
        model.add(LSTM(units=int(params['units']),
                    activation='tanh',
                    recurrent_dropout=params['recurrent_dropout'],
                    return_sequences=False,
                    input_shape=(SEQ_LEN, X_train.shape[-1])))
        model.add(BatchNormalization()),
        model.add(Dense(1, activation='linear'))
        optimizer = AdamW(learning_rate=params['learning_rate'])
        model.compile(optimizer=optimizer, loss=RMSE)
        return model

    # 목적 함수 정의
    def objective(params):
        model = lstm_model(params)
        early_stopping = EarlyStopping(monitor='val_loss', patience=PATIENCE, verbose=1, restore_best_weights=True) # PATIENCE는 현재 30번으로 되어 있으며 30번 학습 뒤로 가장 낮은 loss가 나오지 않으면 조기종료
        history = model.fit(X_train, y_train,
                            epochs=EPOCHS,
                            batch_size=int(params['batch_size']),
                            shuffle=False,
                            validation_data=(X_validation, y_validation),
                            callbacks=[early_stopping],
                            verbose=1)

        # Early stopping이 발생한 시점에서의 validation RMSE를 반환
        val_rmse = early_stopping.best
        return val_rmse

    # 하이퍼파라미터 탐색 범위 설정(아래 값들을 원하는 값으로 수정하시면 됩니다)
    space = {
        'units': hp.quniform('units', 32, 256, 16), # LSTM 노드 수를 32개부터 256개까지 16개씩 증가하여 서치
        'recurrent_dropout': hp.uniform('recurrent_dropout', 0, 0.5), # recurrent_dropout 비율 0부터 0.5까지 연속적인 범위에서 무작위로 선택
        'learning_rate': hp.uniform('learning_rate', 0.0005, 0.05), # learning_rate 0.0005부터 0.1 선택
        'batch_size': hp.quniform('batch_size', 4, 64, 4), # LSTM 노드 수를 4개부터 64개까지 4개씩 증가하여 서치
    }

    # Bayesian optimization 실행(최대 실험 횟수MAX_EVALS는 위에서 30개로 설정되어 있으나, 늘려도 됩니다. 단, 시간이 더 오래 걸립니다.)
    print("최적 하이퍼파라미터 추출을 위한 작업을 진행합니다.")
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=MAX_EVALS)

    # 최적 하이퍼파라미터 출력
    print("최적 하이퍼파라미터:", best)

    # 최적 모델 재학습
    print("최적 하이퍼파라미터로 재학습 진행합니다.")
    best_model = lstm_model(best)
    history =  best_model.fit(X_train, y_train,
                epochs=EPOCHS,
                batch_size=int(best['batch_size']),
                validation_data=(X_validation, y_validation))

    # 최적 모델 저장 경로 설정
    model_save_folder = f'{MODEL_PATH}{key}/'
    os.makedirs(model_save_folder, exist_ok=True) # model 저장 폴더가 없으면 생성
    model_save_path = f'{model_save_folder}best_model.h5'

    # 최적 모델 저장
    print("최적 모델을 저장합니다.")
    save_model(best_model, model_save_path)

    # 그래프 관련 저장 경로 설정
    graph_save_folder = f'{GRAPH_PATH}{key}/'
    os.makedirs(graph_save_folder, exist_ok=True) # model 저장 폴더가 없으면 생성
    loss_graph_save_path = f'{graph_save_folder}loss_graph.png'
    feature_importance_graph_save_path = f'{graph_save_folder}feature_importance_graph.png'
    total_graph_save_path = f'{graph_save_folder}total_graph.png'
    train_graph_save_path = f'{graph_save_folder}train_graph.png'
    validation_graph_save_path = f'{graph_save_folder}validation_graph.png'
    test_graph_save_path = f'{graph_save_folder}test_graph.png'

    # epoch별 train_loss와 validation_loss에 대한 그래프 작성
    print("Loss graph를 작성합니다")
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(loss_graph_save_path)
    plt.show()

    # 모델을 사용하여 예측
    print("train, validation, test set에 대한 예측을 진행합니다.")
    y_train_pred = best_model.predict(X_train)
    y_validation_pred = best_model.predict(X_validation)
    y_test_pred = best_model.predict(X_test)

    df_train_graph = pd.DataFrame({'time':time_train.reshape(-1), 'HOUSING_E_actual':y_train.reshape(-1), 'HOUSING_E_pred':y_train_pred.reshape(-1), 'type':'train'})
    df_validation_graph = pd.DataFrame({'time':time_validation.reshape(-1), 'HOUSING_E_actual':y_validation.reshape(-1), 'HOUSING_E_pred':y_validation_pred.reshape(-1), 'type':'validation'})
    df_test_graph = pd.DataFrame({'time':time_test.reshape(-1), 'HOUSING_E_actual':y_test.reshape(-1), 'HOUSING_E_pred':y_test_pred.reshape(-1), 'type':'test'})
    df_graph = pd.concat([df_train_graph,df_validation_graph,df_test_graph],axis=0).reset_index(drop=True)
    df_graph['time'] = pd.to_datetime(df_graph['time'], format="%Y%m")
    df_graph['time'] = df_graph['time'].dt.strftime('%Y-%m')

    # 2021년 01월 예측
    print("2021년 01월에 대한 예측을 진행합니다.")
    tmp = data_dict[key].iloc[-SEQ_LEN:,:]
    tmp = tmp[INPUT_COLUMN]
    tmp = mm_scaler.transform(tmp)
    tmp = np.expand_dims(tmp,0)
    prediction = best_model.predict(tmp)[0][0]

    # 각 지표 측정
    result_tmp = {}
    result_tmp['ID'] = key
    result_tmp['train_RMSE'] = [np.sqrt(mean_squared_error(y_train, y_train_pred))]
    result_tmp['train_MSE'] = [mean_squared_error(y_train, y_train_pred)]
    result_tmp['train_MAE'] = [mean_absolute_error(y_train, y_train_pred)]
    result_tmp['train_MAPE'] = [np.mean(np.abs((y_train - y_train_pred) / y_train)) * 100]
    result_tmp['validation_RMSE'] = [np.sqrt(mean_squared_error(y_validation, y_validation_pred))]
    result_tmp['validation_MSE'] = [mean_squared_error(y_validation, y_validation_pred)]
    result_tmp['validation_MAE'] = [mean_absolute_error(y_validation, y_validation_pred)]
    result_tmp['validation_MAPE'] = [np.mean(np.abs((y_validation - y_validation_pred) / y_validation)) * 100]
    result_tmp['test_RMSE'] = [np.sqrt(mean_squared_error(y_test, y_test_pred))]
    result_tmp['test_MSE'] = [mean_squared_error(y_test, y_test_pred)]
    result_tmp['test_MAE'] = [mean_absolute_error(y_test, y_test_pred)]
    result_tmp['test_MAPE'] = [np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100]
    result_tmp['next_month_HOUSING_E'] = [prediction]
    result_tmp = pd.DataFrame(result_tmp)
    total_result = pd.concat([total_result, result_tmp], axis=0)

    # 변수 중요도 추출
    print("변수 중요도를 계산합니다")
    print("Original Mean Squared Error:", result_tmp['test_MSE'])

    feature_importance = {}
    feature_importance['ID'] = key
    for i in range(len(INPUT_COLUMN)):
        X_permuted = X_test.copy()
        X_permuted[:,:,i] = shuffle(X_permuted[:,:,i])

        y_pred_permuted = best_model.predict(X_permuted)
        mse_permuted = mean_squared_error(y_test, y_pred_permuted)

        feature_importance[INPUT_COLUMN[i]] = np.abs(result_tmp['test_MSE'][0] - mse_permuted)

    # 중요도 출력
    print("변수 중요도를 출력합니다")
    print("Permutation Feature Importance:", feature_importance)
    plt.barh(list(feature_importance.keys())[1:], list(feature_importance.keys())[1:])
    plt.xlabel('Permutation Feature Importance')
    plt.ylabel('Features')
    plt.title('Permutation Feature Importance for LSTM Model')
    plt.savefig(feature_importance_graph_save_path)
    plt.show()

    feature_importance = pd.DataFrame.from_dict(feature_importance, orient='index', columns=['Permutation Feature Importance'])
    feature_importance = feature_importance.T
    feature_importance.index = [0]
    total_feature_importance = pd.concat([total_feature_importance, feature_importance], axis=0)

    # 전체 예측 그래프
    print("전체 데이터에 대한 실제값과 예측값 비교 그래프를 작성합니다")
    ax1 = sns.lineplot(x='time', y='HOUSING_E_actual', data=df_graph)
    ax1 = sns.lineplot(x='time', y='HOUSING_E_pred', data=df_graph)
    ax1.set_xticks(np.arange(0, len(df_graph)+1,12))
    plt.xticks(rotation=45)
    plt.title('Total actual vs predict')
    plt.xlabel('Time')
    plt.ylabel('HOUSING_E')
    plt.legend()
    plt.savefig(total_graph_save_path)
    plt.show()

    # train 예측 그래프
    print("train set에 대한 실제값과 예측값 비교 그래프를 작성합니다")
    ax2 = sns.lineplot(x='time', y='HOUSING_E_actual', data=df_graph.loc[df_graph['type']=='train'])
    ax2 = sns.lineplot(x='time', y='HOUSING_E_pred', data=df_graph.loc[df_graph['type']=='train'])
    ax2.set_xticks(np.arange(0, len(df_graph.loc[df_graph['type']=='train'])+1,12))
    plt.xticks(rotation=45)
    plt.title('Train actual vs predict')
    plt.xlabel('Time')
    plt.ylabel('HOUSING_E')
    plt.legend()
    plt.savefig(train_graph_save_path)
    plt.show()

    # validaiton 예측 그래프
    print("validaiton set에 대한 실제값과 예측값 비교 그래프를 작성합니다")
    ax3 = sns.lineplot(x='time', y='HOUSING_E_actual', data=df_graph.loc[df_graph['type']=='validation'])
    ax3 = sns.lineplot(x='time', y='HOUSING_E_pred', data=df_graph.loc[df_graph['type']=='validation'])
    ax3.set_xticks(np.arange(0, len(df_graph.loc[df_graph['type']=='validation'])+1,3))
    plt.xticks(rotation=45)
    plt.title('Validation actual vs predict')
    plt.xlabel('Time')
    plt.ylabel('HOUSING_E')
    plt.legend()
    plt.savefig(validation_graph_save_path)
    plt.show()

    # test 예측 그래프
    print("test set에 대한 실제값과 예측값 비교 그래프를 작성합니다")
    ax4 = sns.lineplot(x='time', y='HOUSING_E_actual', data=df_graph.loc[df_graph['type']=='test'])
    ax4 = sns.lineplot(x='time', y='HOUSING_E_pred', data=df_graph.loc[df_graph['type']=='test'])
    ax4.set_xticks(np.arange(0, len(df_graph.loc[df_graph['type']=='test'])+1,3))
    plt.xticks(rotation=45)
    plt.title('Test actual vs predict')
    plt.xlabel('Time')
    plt.ylabel('HOUSING_E')
    plt.legend()
    plt.savefig(test_graph_save_path)
    plt.show()

In [ ]:
# 최종 metric 결과 및 feature importance 결과 저장
os.makedirs(METRIC_RESULT_PATH, exist_ok=True) # model 저장 폴더가 없으면 생성

print("각 행정구역에 대한 feature importance 결과를 저장합니다.")
total_feature_importance.to_csv(METRIC_RESULT_PATH+"feature_importance.csv")

print("각 행정구역에 대한 최종 metric 결과를 저장합니다.")
total_result.to_csv(METRIC_RESULT_PATH+"total_result.csv")